# MCM 2025 Problem C: Olympic Medal Prediction & Analysis

This notebook implements a comprehensive modeling approach for predicting Olympic medal counts, aimed at the "Outstanding Winner" level.

## Project Goals
1.  **Medal Count Model**: Predict 2028 Gold and Total medals with uncertainty/precision estimates.
2.  **2028 Projections**: Identify improvers/decliners for LA 2028.
3.  **First Medal Winners**: Probabilistic model for nations winning their first medal.
4.  **Event Analysis**: Impact of event types and host country selection.
5.  **"Great Coach" Effect**: Detect and quantify the impact of elite coaching.
6.  **Strategic Insights**: Recommendations for Olympic committees.

## 1. Data Loading and Preprocessing

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import PoissonRegressor
from sklearn.model_selection import train_test_split
from scipy import stats

warnings.filterwarnings('ignore')
sns.set_style("whitegrid")

# Define Data Paths
DATA_DIR = '../data/raw/2025_Problem_C_Data'
ATHLETES_FILE = os.path.join(DATA_DIR, 'summerOly_athletes.csv')
HOSTS_FILE = os.path.join(DATA_DIR, 'summerOly_hosts.csv')
MEDALS_FILE = os.path.join(DATA_DIR, 'summerOly_medal_counts.csv')
PROGRAMS_FILE = os.path.join(DATA_DIR, 'summerOly_programs.csv')

# Load Data
def load_csv_safe(filepath):
    try:
        # Try UTF-8 first
        return pd.read_csv(filepath)
    except UnicodeDecodeError:
        # Fallback to ISO-8859-1 (common for legacy datasets with accents)
        return pd.read_csv(filepath, encoding='ISO-8859-1')

try:
    df_athletes = load_csv_safe(ATHLETES_FILE)
    df_hosts = load_csv_safe(HOSTS_FILE)
    df_medals = load_csv_safe(MEDALS_FILE)
    df_programs = load_csv_safe(PROGRAMS_FILE)
    
    print("Data loaded successfully.")
    print(f"Athletes shape: {df_athletes.shape}")
    print(f"Hosts shape: {df_hosts.shape}")
    print(f"Medals shape: {df_medals.shape}")
    print(f"Programs shape: {df_programs.shape}")
except Exception as e:
    print(f"Error loading data: {e}")
    print("Please check the 'DATA_DIR' path.")

Data loaded successfully.
Athletes shape: (252565, 9)
Hosts shape: (35, 2)
Medals shape: (1435, 7)
Programs shape: (74, 35)


In [7]:
# Display first few rows to understand structure
display(df_medals.head())
display(df_hosts.head())
display(df_programs.head())

# Check for consistency in Country names
# Note: The column is likely 'NOC' (National Olympic Committee) based on standard Olympic data formats
print(f"Columns in Medal Table: {df_medals.columns.tolist()}")
if 'Team' in df_medals.columns:
    print(f"Unique Teams in Medal Table: {df_medals['Team'].nunique()}")
elif 'NOC' in df_medals.columns:
    print(f"Unique Teams in Medal Table: {df_medals['NOC'].nunique()}")
else:
    print("Could not find Team/NOC column.")

,Rank,NOC,Gold,Silver,Bronze,Total,Year
0,1,United States,11,7,2,20,1896
1,2,Greece,10,18,19,47,1896
2,3,Germany,6,5,2,13,1896
3,4,France,5,4,2,11,1896
4,5,Great Britain,2,3,2,7,1896


,Rank,NOC,Gold,Silver,Bronze,Total,Year
0,1,United States,11,7,2,20,1896
1,2,Greece,10,18,19,47,1896
2,3,Germany,6,5,2,13,1896
3,4,France,5,4,2,11,1896
4,5,Great Britain,2,3,2,7,1896


,Year,Host
0,1896,"Athens, Greece"
1,1900,"Paris, France"
2,1904,"St. Louis, United States"
3,1908,"London, United Kingdom"
4,1912,"Stockholm, Sweden"


,Sport,Discipline,Code,Sports Governing Body,1896,1900,1904,1906*,1908,1912,...,1988,1992,1996,2000,2004,2008,2012,2016,2020,2024
0,Aquatics,Artistic Swimming,SWA,World Aquatics,0,0,0,0,0,0,...,2,2,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
1,Aquatics,Diving,DIV,World Aquatics,0,0,2,1,2,4,...,4,4,4.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0
2,Aquatics,Marathon Swimming,OWS,World Aquatics,0,0,0,0,0,0,...,0,0,0.0,0.0,0.0,2.0,2.0,2.0,2.0,2.0
3,Aquatics,Swimming,SWM,World Aquatics,4,7,9,4,6,9,...,31,31,32.0,32.0,32.0,32.0,32.0,32.0,35.0,35.0
4,Aquatics,Water Polo,WPO,World Aquatics,0,1,1,0,1,1,...,1,1,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0


Columns in Medal Table: ['Rank', 'NOC', 'Gold', 'Silver', 'Bronze', 'Total', 'Year']
Unique Teams in Medal Table: 210


## 2. Feature Engineering

We need to construct a dataset suitable for regression.
**Target Variables:** `Gold`, `Total`
**Features:**
*   `Year` (Trend)
*   `Lagged_Medals` (Performance in previous Olympics)
*   `Is_Host` (Binary: 1 if host, 0 otherwise)
*   `Total_Events` (Number of events in that year)
*   `Prev_Host` (Did they host the previous one? Momentum)
*   `GDP`, `Population` (External data - *Note: You might need to import external datasets for this, or use proxies like team size*)

```python
# Placeholder for Feature Engineering Code
# 1. Merge Hosts to get Host Country
# 2. Create 'Is_Host' flag
# 3. Calculate Lagged Features (e.g., Gold_t-1)
# 4. Count events per year from df_programs and merge
```

## 3. Model Development (Medal Counts)

We will use a **Negative Binomial Regression** or **Poisson Regression** since medal counts are count data and likely overdispersed (variance > mean).

$$ \ln(E[Y]) = \beta_0 + \beta_1 X_1 + \dots + \beta_k X_k $$

## 4. 2028 Predictions (Los Angeles)

*   Set up the input vector for 2028 (Host=USA, Year=2028, Lagged=2024 results).
*   Predict Gold and Total.
*   Generate Prediction Intervals (using bootstrap or model distribution).

## 5. "Great Coach" Effect Analysis

**Hypothesis:** A "Great Coach" effect is observable as a *structural break* or significant *level shift* in a specific sport's medal count for a country, which cannot be explained by the country's general trend.

**Methodology:**
1.  Isolate data for specific Country-Sport pairs (e.g., USA-Gymnastics, China-Volleyball).
2.  Use Change Point Detection (e.g., CUSUM of medals/rank).
3.  Correlate detected jumps with known coaching changes if possible, or quantify the magnitude of unexplained jumps.

## 6. Strategic Insights
*   **First Medal Probability**: Zero-Inflated Model for countries with 0 medals.
*   **Event Importance**: Correlation between specific sport event counts and total medals.
